<a href="https://colab.research.google.com/github/Vishwajeet0830/AI-Travel-Agent/blob/main/TravelMultiModalAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anthropic langchain langgraph langchain_anthropic streamlit streamlit-chat pydantic python-dotenv

In [2]:
%%writefile travel_planner.py
import os
import streamlit as st
from anthropic import Anthropic
from typing import Dict, List, Any, Optional, TypedDict
from pydantic import BaseModel, Field

# Initialize the Anthropic client with the environment variable
api_key = os.environ.get('ANTHROPIC_API_KEY')
if api_key:
    anthropic_client = Anthropic(api_key=api_key)
else:
    # This will be caught when running the script directly
    anthropic_client = None

# Agent functions
def itinerary_planner_agent(user_input):
    """Creates an initial travel itinerary"""
    prompt = f"""
    You are an expert travel itinerary planner. Create a detailed day-by-day itinerary based on these preferences:

    - Origin: {user_input['departure_location']}
    - Destination: {user_input['destination']}
    - Travel Month: {user_input['travel_month']}
    - Number of Days: {user_input['travel_days']}
    - Number of Travelers: {user_input['people_count']}
    - Traveler Type: {user_input['audience_choice']}
    - Budget: ${user_input['budget']} USD

    Format your response as a structured itinerary with day-by-day breakdowns.
    """

    response = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=2000,
        temperature=0.7,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

def transportation_agent(user_input):
    """Suggests transportation options"""
    prompt = f"""
    You are a transportation specialist. Suggest 3 realistic transportation options from
    {user_input['departure_location']} to {user_input['destination']} for {user_input['people_count']} people
    traveling in {user_input['travel_month']} for a {user_input['audience_choice']} type trip.

    For each option, include type, estimated cost, travel time, and pros/cons.
    """

    response = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0.5,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

def accommodation_agent(user_input):
    """Suggests accommodation options"""
    prompt = f"""
    You are an accommodation specialist. Suggest 3 realistic accommodation options in
    {user_input['destination']} for {user_input['people_count']} people traveling in {user_input['travel_month']}
    for a {user_input['audience_choice']} type trip with a budget of ${user_input['budget']} for {user_input['travel_days']} days.

    For each option, include type, location, approximate cost per night, and key amenities.
    """

    response = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0.5,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

def local_information_agent(user_input):
    """Provides local information"""
    prompt = f"""
    You are a local information specialist. Provide essential information for travelers visiting
    {user_input['destination']} in {user_input['travel_month']}, including:

    1. Typical weather conditions
    2. Local customs and etiquette
    3. Safety considerations
    4. Transportation options within the destination
    5. Any special events occurring during that month
    """

    response = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1200,
        temperature=0.5,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

def budget_analyzer_agent(user_input):
    """Analyzes and optimizes budget"""
    prompt = f"""
    You are a travel budget specialist. Analyze and optimize a ${user_input['budget']} budget for
    {user_input['people_count']} people traveling to {user_input['destination']} for {user_input['travel_days']} days
    in {user_input['travel_month']}.

    Provide:
    1. Suggested budget breakdown by category (accommodation, transportation, food, activities, etc.)
    2. Money-saving tips specific to this destination
    3. Estimate of total costs and whether the budget is realistic
    """

    response = anthropic_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1200,
        temperature=0.3,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

# Function to run the travel planner
def run_travel_planner(user_input):
    """Run all agents sequentially"""
    results = {
        "user_input": user_input,
        "errors": []
    }

    try:
        print("Starting itinerary planning...")
        results["itinerary"] = itinerary_planner_agent(user_input)

        print("Planning transportation...")
        results["transportation_options"] = transportation_agent(user_input)

        print("Finding accommodations...")
        results["accommodation_options"] = accommodation_agent(user_input)

        print("Gathering local information...")
        results["local_info"] = local_information_agent(user_input)

        print("Analyzing budget...")
        results["budget_analysis"] = budget_analyzer_agent(user_input)

        print("Planning complete!")
    except Exception as e:
        print(f"Error in travel planning: {str(e)}")
        results["errors"].append(str(e))

    return results

# Create a simple streamlit app
def create_streamlit_app():
    """Create a Streamlit app for the travel planner"""

    st.write("""
    # AI Travel Planner

    Plan your perfect trip with our multi-agent AI travel planning system!
    """)

    # Get user input
    with st.form("travel_form"):
        st.write("## Trip Details")
        col1, col2 = st.columns(2)

        with col1:
            departure_location = st.text_input("Departure Location (City, Country)", "New York, USA")
            destination = st.text_input("Destination (City, Country)", "Paris, France")
            people_count = st.number_input("Number of Travelers", min_value=1, value=2)

        with col2:
            travel_days = st.slider("Travel Days", min_value=1, max_value=30, value=7)
            travel_month = st.selectbox(
                "Travel Month",
                ["January", "February", "March", "April", "May", "June",
                 "July", "August", "September", "October", "November", "December"]
            )
            audience_choice = st.selectbox(
                "Trip Type",
                ["Young at Heart Adventure", "Family Friendly with Kids", "Elderly Inclusive"]
            )

        budget = st.number_input("Budget (USD)", min_value=500, value=5000)
        submitted = st.form_submit_button("Plan My Trip")

    if submitted:
        # Collect user input
        user_input = {
            "departure_location": departure_location,
            "destination": destination,
            "people_count": people_count,
            "travel_days": travel_days,
            "travel_month": travel_month,
            "audience_choice": audience_choice,
            "budget": budget
        }

        with st.spinner("Planning your trip... This may take a minute or two..."):
            # Run the travel planner
            final_state = run_travel_planner(user_input)

            # Display results in tabs
            tab1, tab2, tab3, tab4, tab5 = st.tabs(["Itinerary", "Transportation", "Accommodation", "Local Info", "Budget"])

            with tab1:
                st.write("## Your Travel Itinerary")
                st.write(final_state.get("itinerary", "No itinerary generated"))

            with tab2:
                st.write("## Transportation Options")
                st.write(final_state.get("transportation_options", "No transportation options generated"))

            with tab3:
                st.write("## Accommodation Options")
                st.write(final_state.get("accommodation_options", "No accommodation options generated"))

            with tab4:
                st.write("## Local Information")
                st.write(final_state.get("local_info", "No local information generated"))

            with tab5:
                st.write("## Budget Analysis")
                st.write(final_state.get("budget_analysis", "No budget analysis generated"))

            # Check for errors
            if final_state.get("errors"):
                st.error("Errors occurred during planning:")
                for error in final_state.get("errors", []):
                    st.write(f"- {error}")

Overwriting travel_planner.py


In [3]:
%%writefile gradio_app.py
import os
import gradio as gr
import sys

# Get API key from command line argument
if len(sys.argv) > 1:
    api_key = sys.argv[1]
    os.environ['ANTHROPIC_API_KEY'] = api_key
else:
    api_key = os.environ.get('ANTHROPIC_API_KEY')
    if not api_key:
        print("Error: No API key provided")
        sys.exit(1)

from travel_planner import run_travel_planner, anthropic_client

def plan_trip(departure_location, destination, people_count, travel_days, travel_month, audience_choice, budget):
    user_input = {
        "departure_location": departure_location,
        "destination": destination,
        "people_count": int(people_count),
        "travel_days": int(travel_days),
        "travel_month": travel_month,
        "audience_choice": audience_choice,
        "budget": float(budget)
    }

    # Show processing message
    print(f"Planning trip from {departure_location} to {destination}...")

    # Run the travel planner
    result = run_travel_planner(user_input)

    # Return results for each tab
    return (
        result["itinerary"] or "Itinerary not generated",
        result["transportation_options"] or "Transportation options not generated",
        result["accommodation_options"] or "Accommodation options not generated",
        result["local_info"] or "Local information not generated",
        result["budget_analysis"] or "Budget analysis not generated"
    )

# Create the Gradio interface
with gr.Blocks(title="AI Travel Planner") as demo:
    gr.Markdown("# AI Travel Planner")
    gr.Markdown("Plan your perfect trip with our multi-agent AI travel planning system!")

    with gr.Row():
        with gr.Column():
            departure = gr.Textbox(label="Departure Location", value="New York, USA")
            destination = gr.Textbox(label="Destination", value="Paris, France")
            people = gr.Number(label="Number of Travelers", value=2, minimum=1)
            days = gr.Slider(label="Travel Days", minimum=1, maximum=30, value=7, step=1)

        with gr.Column():
            month = gr.Dropdown(
                label="Travel Month",
                choices=["January", "February", "March", "April", "May", "June",
                        "July", "August", "September", "October", "November", "December"],
                value="June"
            )
            trip_type = gr.Dropdown(
                label="Trip Type",
                choices=["Young at Heart Adventure", "Family Friendly with Kids", "Elderly Inclusive"],
                value="Young at Heart Adventure"
            )
            budget = gr.Number(label="Budget (USD)", value=5000, minimum=500)

    # Add a button to trigger the planning process
    plan_button = gr.Button("Plan My Trip")

    # Create output tabs for different aspects of the trip
    with gr.Tabs():
        with gr.TabItem("Itinerary"):
            itinerary_output = gr.Markdown()
        with gr.TabItem("Transportation"):
            transport_output = gr.Markdown()
        with gr.TabItem("Accommodation"):
            accommodation_output = gr.Markdown()
        with gr.TabItem("Local Info"):
            local_info_output = gr.Markdown()
        with gr.TabItem("Budget"):
            budget_output = gr.Markdown()

    # Connect the button to the plan_trip function
    plan_button.click(
        fn=plan_trip,
        inputs=[departure, destination, people, days, month, trip_type, budget],
        outputs=[itinerary_output, transport_output, accommodation_output, local_info_output, budget_output]
    )

if __name__ == "__main__":
    demo.launch(share=True)

Overwriting gradio_app.py


In [4]:
from google.colab import userdata
import os

# Get API key from userdata
api_key = userdata.get('ANTHROPIC_API_KEY')
# Set as environment variable
os.environ['ANTHROPIC_API_KEY'] = api_key

print(f"API key set: {api_key[:5]}..." if api_key else "API key not found")

API key set: sk-an...


In [ ]:
# Install required packages
!pip install -q anthropic langchain langgraph langchain_anthropic gradio pydantic

# Run the Gradio app, passing the API key as an argument
!python gradio_app.py "$ANTHROPIC_API_KEY"

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://be84373f88b449149e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Planning trip from New York, USA to Banglore, India...
Starting itinerary planning...
Planning transportation...
Finding accommodations...
Gathering local information...
Analyzing budget...
Planning complete!
